# Real Traces Plotting/Analysis

Traces got from [GitHub](https://github.com/Azure/AzurePublicDataset/blob/master/AzureFunctionsDataset2019.md). See `dataset/REDAME.md`.

In [ ]:
# Common imports.
from pathlib import Path

%matplotlib ipympl
import base

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

Global options of the notebook:

* `data_file`: the full path of the data (a CSV file)

In [ ]:
data_file = Path("/home/emanuele/marl-dfaas/dataset/data/invocations_per_function_md.anon.d01.csv")

## Structure of a invocation file

In [ ]:
invocations = pd.read_csv(data_file)  # Read the data, takes time.

In [ ]:
print("Shape (rows, columns) =", invocations.shape)
print("First 10 columns =", list(invocations.columns[:10]))
print("Last 10 columns =", list(invocations.columns[-10:]))

The first three columns are hashes, consistent between all files:

* `HashOwner`: owner of the application. On owner can have multiple applications.
* `HashApp`: application. An application can have only one owner but many functions. Note two identical application (and functions) have two different hashes since they belong to different owners.
* `HashFunction`: the single function.
* `Trigger`: what causes the function execution.

The remaining columns are how many invocations there were for each minute in a single 24-hours day.

## Plot of a generic trace

Plots a single trace from the given data file. The trace can be selected by its hash (full or partial) or by the index inside the data file.

In [ ]:
function_hash = "115ca7a2b5bc290052c3da74cd0347d19c3c67b7d5aa66e9a975e427f25fc7ed"
function_index = 101
select_by_hash = False

if select_by_hash:
    trace = invocations[invocations["HashFunction"].str.contains(function_hash)]
else:
    trace = invocations.iloc[[function_index]]

assert trace.shape[0] == 1, "Trace must be unique"

# Extract the first columns and the invocations columns ("1" -> "1440")
owner, app, func, trigger = trace.iloc[0][:"Trigger"]
invocs = trace.iloc[0]["1":]

In [ ]:
print("Owner =", owner)
print("Application =", app)
print("Function =", func)
print("Trigger =", trigger)

In [ ]:
fig = plt.figure(layout="constrained")
fig.canvas.header_visible = False
ax = fig.subplots()

minutes_idx = np.arange(1, len(invocs)+1)
ax.bar(minutes_idx, invocs)

ax.set_title(f"Function invocations (function = {func[:10]})")
ax.set_ylabel("Invocations")
ax.set_xlabel("Minute")

ax.grid(axis="both")
ax.set_axisbelow(True)  # By default the axis is over the content.